<a href="https://colab.research.google.com/github/tasosnikitakis/Data_Science_Notebooks/blob/main/pharmacy_social_security_invoices_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.0
    Uninstalling Werkzeug-3.0.0:
      Successfully uninstalled Werkzeug-3.0.0


In [2]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import plotly.express as ptx
import plotly.graph_objects as go

#Dataframe Creation

In [25]:
social_security_invoices_df = pd.read_excel("/content/drive/MyDrive/pharmacy_data/Receivable Invoices.xlsx")

In [4]:
social_security_invoices_df.head()

,DATE,SOCIAL SECURITY PAYER,INVOICE
0,31/01/2019,ΕΔΟΕΑΠ,26.58
1,31/01/2019,ΤΥΠΕΤ,14.52
2,31/01/2019,ΕΟΠΥΥ,14678.16
3,31/01/2019,ΤΕΑΠΑΣΑ,17.59
4,31/01/2019,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,583.64


##Data Exploration

In [27]:
social_security_invoices_df.columns

Index(['DATE', 'SOCIAL SECURITY PAYER', 'INVOICE'], dtype='object')

##Data Cleaning

In [28]:
social_security_invoices_df.DATE = pd.to_datetime(social_security_invoices_df.DATE, format="%d/%m/%Y")
social_security_invoices_df.style.format({"DATE": lambda t: t.strftime("%d/%m/%Y")})

,DATE,SOCIAL SECURITY PAYER,INVOICE
0,31/01/2019,ΕΔΟΕΑΠ,26.580000
1,31/01/2019,ΤΥΠΕΤ,14.520000
2,31/01/2019,ΕΟΠΥΥ,14678.160000
3,31/01/2019,ΤΕΑΠΑΣΑ,17.590000
4,31/01/2019,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,583.640000
5,28/02/2019,ΕΟΠΥΥ,13818.940000
6,28/02/2019,ΤΕΑΠΑΣΑ,5.690000
7,28/02/2019,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,237.660000
8,31/03/2019,ΤΥΠΕΤ,17.360000
9,31/03/2019,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,176.620000


##Aggregation function by year and month creating dataframe with all monthly social security invoices sums

In [7]:
def aggregate_by_year_month_sums(social_security_invoices_df):
  # Assuming you have your dataframe named 'df'
  # First, convert the 'DATE' column to a datetime format if it's not already
  social_security_invoices_df['DATE'] = pd.to_datetime(social_security_invoices_df['DATE'], format='%d/%m/%Y')

  # Extract the year and month from the 'DATE' column
  social_security_invoices_df['YEAR'] = social_security_invoices_df['DATE'].dt.year
  social_security_invoices_df['MONTH'] = social_security_invoices_df['DATE'].dt.month

  # Group by 'YEAR', 'MONTH' and sum the 'EXPENSE' column
  aggregated_df = social_security_invoices_df.groupby(['YEAR', 'MONTH'])['INVOICE'].sum().reset_index()

  # Display the aggregated dataframe
  return aggregated_df

In [8]:
social_security_invoices_monthly_sums_df = aggregate_by_year_month_sums(social_security_invoices_df)

In [9]:
social_security_invoices_monthly_sums_df

,YEAR,MONTH,INVOICE
0,2019,1,15320.49
1,2019,2,14062.29
2,2019,3,14216.84
3,2019,4,14738.65
4,2019,5,16470.77
5,2019,6,12378.31
6,2019,7,14780.57
7,2019,8,8547.81
8,2019,9,11550.77
9,2019,10,17290.62


##Function that takes dataframe, year and social security payer and returns dataframe with monthly values of sums of invoices for payer

In [10]:
def filter_by_year_payer_monthly_sums(df, year, social_security_payer):
    # Filter the dataframe by year and social security payer
    filtered_df = df[(df['DATE'].dt.year == year) & (df['SOCIAL SECURITY PAYER'] == social_security_payer)]

    # Group the filtered dataframe by month and calculate the sum of 'INVOICE' column
    grouped_df = filtered_df.groupby(df['DATE'].dt.month)['INVOICE'].sum().reset_index()

    # Rename columns for clarity
    grouped_df.columns = ['Month', 'Monthly Sum']
    return grouped_df

In [11]:
year = 2019
social_security_payer = 'ΕΟΠΥΥ'
result_df = filter_by_year_payer_monthly_sums(social_security_invoices_df, year, social_security_payer)
print(result_df)

    Month  Monthly Sum
0       1     14678.16
1       2     13818.94
2       3     13969.61
3       4     14104.80
4       5     16066.59
5       6     12071.41
6       7     14464.31
7       8      8254.93
8       9     11356.98
9      10     16874.81
10     11     15763.74
11     12     14062.28


##Function that groups payers invoices by year and returns dataframe

In [12]:
def group_by_year(input_df, year):
    # Filter the dataframe for the specified year
    filtered_df = input_df[input_df['DATE'].dt.year == year]

    # Group the filtered dataframe by 'SOCIAL SECURITY PAYER' and calculate the sum of 'INVOICE' column
    grouped_df = filtered_df.groupby('SOCIAL SECURITY PAYER')['INVOICE'].sum().reset_index()

    # Rename columns for clarity
    grouped_df.columns = ['SOCIAL SECURITY PAYER', 'Total Sum']

    return grouped_df

In [13]:
year_to_group = 2019
result_df = group_by_year(social_security_invoices_df, year_to_group)
print(result_df)

  SOCIAL SECURITY PAYER  Total Sum
0                ΕΔΟΕΑΠ      26.58
1                 ΕΟΠΥΥ  165486.56
2       ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ    3587.82
3               ΤΕΑΠΑΣΑ     159.21
4                 ΤΥΠΕΤ     395.94


##Month over Month change rate function. The function takes as arguments the social security payer and returns a dataframe that contains DATE (in MM/YYYY format), SOCIAL SECURITY PAYER, Invoices Sum (That contains the summed valued of invoices for the payer for the month), Change (The percentage change over the previous month), Abs Change (The absolut change over the previous month)

In [14]:
def calculate_mom_change(df, social_security_payer):
    # Filter the dataframe for the specified social security payer
    filtered_df = df[df['SOCIAL SECURITY PAYER'] == social_security_payer].copy()

    # Calculate the MoM change and absolute change
    filtered_df['Change'] = filtered_df['INVOICE'].pct_change() * 100
    filtered_df['Abs Change'] = filtered_df['INVOICE'].diff()

    # Reorder the columns
    filtered_df = filtered_df[['DATE', 'SOCIAL SECURITY PAYER', 'INVOICE', 'Change', 'Abs Change']]

    return filtered_df

In [15]:
social_security_payer = 'ΕΟΠΥΥ'
result_df = calculate_mom_change(social_security_invoices_df, social_security_payer)

In [16]:
result_df

,DATE,SOCIAL SECURITY PAYER,INVOICE,Change,Abs Change
2,2019-01-31,ΕΟΠΥΥ,14678.16,NaN,NaN
5,2019-02-28,ΕΟΠΥΥ,13818.94,-5.853731,-859.22
10,2019-03-31,ΕΟΠΥΥ,13969.61,1.090315,150.67
14,2019-04-30,ΕΟΠΥΥ,14104.80,0.967744,135.19
17,2019-05-31,ΕΟΠΥΥ,16066.59,13.908669,1961.79
21,2019-06-30,ΕΟΠΥΥ,12071.41,-24.866384,-3995.18
23,2019-07-31,ΕΟΠΥΥ,14464.31,19.822871,2392.90
27,2019-08-31,ΕΟΠΥΥ,8254.93,-42.928975,-6209.38
31,2019-09-30,ΕΟΠΥΥ,11356.98,37.578150,3102.05
35,2019-10-31,ΕΟΠΥΥ,16874.81,48.585363,5517.83


In [17]:
df = social_security_invoices_df.query("YEAR == 2020")
fig = ptx.pie(df, values='INVOICE', names='SOCIAL SECURITY PAYER', title='2020 Social Security Payers Invoices Pie Chart')
fig.show()

In [18]:
def calculate_year_over_year_change_rate(df, social_security_payer):
    df_copy = df.copy()
    # Filter the dataframe for the specified social security payer
    df = df[df['SOCIAL SECURITY PAYER'] == social_security_payer].copy()

    # Assuming 'DATE' is a string, let's convert it to datetime for better handling
    df['DATE'] = pd.to_datetime(df['DATE'], format='%d/%m/%Y')

    # Extract the year and month from the 'DATE' column
    df['Year'] = df['DATE'].dt.year
    df['Month'] = df['DATE'].dt.month

    # Initialize a list to store the calculated YoY change rates
    yoy_change_rates = []

    # Iterate through the DataFrame rows
    for index, row in df.iterrows():
        current_year = row['Year']
        current_month = row['Month']
        current_value = row['INVOICE']

        # Check if there's a previous year in the DataFrame
        if current_year - 1 in df['Year'].values:
            # Get the previous year's value for the same month
            previous_year_value = df[(df['Year'] == current_year - 1) & (df['Month'] == current_month)]['INVOICE'].values[0]

            # Calculate the YoY change rate
            yoy_change_rate = ((current_value - previous_year_value) / previous_year_value) * 100
        else:
            # No previous year data, set the YoY change rate to NaN
            yoy_change_rate = None

        # Append the calculated YoY change rate to the list
        yoy_change_rates.append(yoy_change_rate)

    # Create a new column 'YoY_Change_Rate' in the DataFrame
    df['YoY_Change_Rate'] = yoy_change_rates
    df.drop(['YEAR', 'MONTH', 'Year', 'Month'], axis=1, inplace=True)
    # Convert the 'DATE' column to a datetime object
    df['DATE'] = pd.to_datetime(df['DATE'])
    # Format the 'DATE' column to 'Month Year' format
    df['DATE'] = df['DATE'].dt.strftime('%B %Y')
    return df

In [19]:
social_security_payer = 'ΕΟΠΥΥ'
yoy_df = calculate_year_over_year_change_rate(social_security_invoices_df, social_security_payer)

In [20]:
yoy_df

,DATE,SOCIAL SECURITY PAYER,INVOICE,YoY_Change_Rate
2,January 2019,ΕΟΠΥΥ,14678.16,NaN
5,February 2019,ΕΟΠΥΥ,13818.94,NaN
10,March 2019,ΕΟΠΥΥ,13969.61,NaN
14,April 2019,ΕΟΠΥΥ,14104.80,NaN
17,May 2019,ΕΟΠΥΥ,16066.59,NaN
21,June 2019,ΕΟΠΥΥ,12071.41,NaN
23,July 2019,ΕΟΠΥΥ,14464.31,NaN
27,August 2019,ΕΟΠΥΥ,8254.93,NaN
31,September 2019,ΕΟΠΥΥ,11356.98,NaN
35,October 2019,ΕΟΠΥΥ,16874.81,NaN


##Function that gets dataframe, year, month and social_security_payer and returns invoice value

In [29]:
social_security_invoices_df['DATE'] = pd.to_datetime(social_security_invoices_df['DATE'], format='%d/%m/%Y')

In [30]:
social_security_invoices_df

,DATE,SOCIAL SECURITY PAYER,INVOICE
0,2019-01-31,ΕΔΟΕΑΠ,26.58
1,2019-01-31,ΤΥΠΕΤ,14.52
2,2019-01-31,ΕΟΠΥΥ,14678.16
3,2019-01-31,ΤΕΑΠΑΣΑ,17.59
4,2019-01-31,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,583.64
...,...,...,...
293,2023-07-30,ΕΟΠΥΥ,9694.89
294,2023-08-30,ΤΥΠΕΤ,115.57
295,2023-08-30,ΕΔΟΕΑΠ,247.14
296,2023-08-30,ΕΟΠΥΥ ΑΝΑΛΩΣΙΜΑ,114.44


In [31]:
def get_invoice_value(df, year, month, social_security_payer):
    # Create a datetime object for the target year and month
    target_date = pd.to_datetime(f'{year}-{month:02d}-01')

    # Convert the 'SOCIAL SECURITY PAYER' column values to strings
    df['SOCIAL SECURITY PAYER'] = df['SOCIAL SECURITY PAYER'].astype(str)

    # Filter the DataFrame based on year, month, and social security payer
    filtered_df = df[(df['DATE'].dt.year == target_date.year) &
                      (df['DATE'].dt.month == target_date.month) &
                      (df['SOCIAL SECURITY PAYER'] == social_security_payer)]

    # Check if there are any matching rows
    if not filtered_df.empty:
        invoice_value = filtered_df['INVOICE'].sum()
        return invoice_value
    else:
        return None  # Return None if no matching rows found


In [32]:
year = 2023
month = 8
social_security_payer = 'ΕΟΠΥΥ'
result = get_invoice_value(social_security_invoices_df, year, month, social_security_payer)

In [33]:
result

6366.5

In [36]:
# Create a Dash web application
app = Dash(__name__)


# Convert the 'DATE' column to datetime
social_security_invoices_df['DATE'] = pd.to_datetime(social_security_invoices_df['DATE'], format='%d/%m/%Y')

# Define the layout of the web application
app.layout = html.Div([
    html.H1("Invoice Value Dashboard"),

    # Input fields for year, month, and social security payer
    dcc.Input(id='year-input', type='number', placeholder='Year'),
    dcc.Input(id='month-input', type='number', placeholder='Month'),
    dcc.Input(id='payer-input', type='text', placeholder='Social Security Payer'),

    # Button to trigger calculation
    html.Button('Calculate', id='calculate-button', n_clicks=0),

    # Card component to display the result
    dcc.Card([
        html.Div(id='result-text')
    ])
])

# Define a callback function to calculate and update the result
@app.callback(
    Output('result-text', 'children'),
    [Input('calculate-button', 'n_clicks')],
    [dash.dependencies.State('year-input', 'value'),
     dash.dependencies.State('month-input', 'value'),
     dash.dependencies.State('payer-input', 'value')]
)
def update_result(n_clicks, year, month, payer):
    if n_clicks > 0:
        result = get_invoice_value(social_security_invoices_df, year, month, payer)
        if result is not None:
            return f'Total Invoice Value for {payer} in {month}/{year}: {result:.2f}'
        else:
            return f'No data found for {payer} in {month}/{year}'
    else:
        return ''

if __name__ == '__main__':
    app.run_server(debug=True)




AttributeError: ignored